In [69]:
import numpy as np
import pickle
import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

Итак, мы хотим сравнить различные модели генерации эмбеддингов по качеству. Можно выбрать следующую метрику. Возьмем одну фотографию. Отсортируем все эмбеддинги по расстоянию до выбранного эмбеддинга. Далее мы можем использовать метрику качества ранжирования(например ndcg), при этом считая релевантными объекты, которые имеют тот же identity label. Такую метрику можно померить на некотором подмножестве объектов(на всех будет долго очень), общем для обоих моделей и усреднить.

Для эксперимента возьмем обученные эмбеддинги от модели facenet, и от самостоятельно обученной модели, а также нам понадобятся identity labels.

In [2]:
embs_facenet = pickle.load(open('app/index/embeddings', 'rb'))

In [3]:
embs_torch = pickle.load(open('app/index/embeddings_torch', 'rb'))

In [4]:
with open('../eval/identity_CelebA.txt') as f:
    labels = np.array([int(line.rstrip().split(' ')[1]) for line in f])

In [5]:
embs_facenet.shape, embs_torch.shape, labels.shape

((202599, 128), (202599, 128), (202599,))

In [11]:
def normalize(embeddings):
    norms = np.sqrt((embeddings**2).sum(axis=1))
    return (embeddings.T / norms).T

def distances(emb, embeddings):
    return np.sqrt(np.sum((embeddings - emb) ** 2, axis=1))

In [10]:
embs_facenet = normalize(embs_facenet)
embs_torch = normalize(embs_torch)

In [51]:
def ndcg(idx, embeddings, labels):
    emb = embeddings[idx]
    dists = distances(emb, embeddings)
    ranks = np.argsort(dists)
    relevant_ranks = ranks[labels == labels[idx]][1:]
    n_rel = len(relevant_ranks)
    dcg = np.sum(1 / np.log2(1 + relevant_ranks))
    idcg = np.sum(1 / np.log2(2 + np.arange(n_rel)))
    return dcg / idcg

In [60]:
test_objects = np.random.randint(0, len(labels), 1000)

In [61]:
facenet_scores = [ndcg(idx, embs_facenet, labels) for idx in tqdm.tqdm(test_objects)]

100%|██████████| 1000/1000 [03:18<00:00,  5.04it/s]


In [62]:
mynet_scores = [ndcg(idx, embs_torch, labels) for idx in tqdm.tqdm(test_objects)]

100%|██████████| 1000/1000 [03:11<00:00,  5.23it/s]


In [64]:
print(np.array(facenet_scores).mean(), np.array(mynet_scores).mean())

0.1834445546168095 0.1836680363338563


Кажется на первый взгляд интересным, что числа такие близкие. Но можно проделать следующий эксперимент. Просто отобразим ранги объектов с одинаковыми метками

In [67]:
emb = embs_facenet[0]
dists = distances(emb, embs_facenet)
ranks = np.argsort(dists)
relevant_ranks = ranks[labels == labels[0]][1:]
print(sorted(relevant_ranks))

[5393, 8073, 13822, 17066, 18980, 26530, 27237, 44372, 55175, 61702, 67579, 79843, 81527, 82578, 86420, 91984, 95215, 112601, 113152, 138033, 139892, 156465, 158219, 172411, 178948, 182755, 187200, 188604, 200096]


Видим, что идентичность определяется не так хорошо. Возможно дело в том, что датасет достаточно шумный и фотографии действительно очень разные. Я сам эту идентичность на глаз плохо определяю. Это примерно согласуется с тем, как я смотрел выдачи. Достаточно редко он выдает одного и того же человека, но при этом все в выдач чем-то похожи.

Хотя интересно, что наша сеть получилась чуть выше по этой метрике.